In [1]:
import requests
import csv
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import os, glob
import time
import pandas as pd
import numpy as np
from vzorci_in_pomozne_definicije import *
    
# Tole dvoje je treba, da se Chrome sproti ne odpira
chrome_options = Options()
chrome_options.add_argument("--headless")
service_obj = Service("chromedriver.exe")

podatki_s_spletnih_strani_se_niso_lokalno_shranjeni = False

In [2]:
# Spodnjo kocko poženemo samo v primeru, da podatki še niso lokalno shranjeni (oz. če spremenimo vrednost bool spremenljivke)
if podatki_s_spletnih_strani_se_niso_lokalno_shranjeni:
    driver = webdriver.Chrome(service = service_obj, options=chrome_options)
    PRVI_SET = 1
    ZADNJI_SET = 1300

    #indeksi_setov_ki_obstajajo = []

    for st_strani in range(PRVI_SET, ZADNJI_SET + 1):
        url = (
            f'https://www.mtgstocks.com/sets/{st_strani}' 
        )
        print(f"\n Zajemam {url}\n")
        driver.get(url)
        time.sleep(1)
        
        
        vsebina = driver.page_source
        naslov_strani = driver.title
        
        stevilo_podstrani_s_kartami = vsebina.count("pagination-page page-item")
        st_besed_Market_na_strani = vsebina.count(">Market<")
        
        print("Naslov strani:", naslov_strani)
        
        if naslov_strani in ["File not found - MTGStocks"]:
            print(f'\n Stran št. {st_strani} ni dosegljiva.\n')
        else:
            print(f'\n Stran št. {st_strani} JE dosegljiva.\n')
            if "Art Series:" in naslov_strani or "From the Vault:" in naslov_strani or st_besed_Market_na_strani == 0:
                continue
            else:
                with open(pridobi_ustrezno_ime_lokalne_datoteke(st_strani), 'w', encoding = "utf-8") as dat:
                    driver.find_element(by=By.XPATH, value = "//*[text() = 'Market']//..").click()
                    for podstran in range(1, stevilo_podstrani_s_kartami + 1):
                        driver.find_element(by=By.XPATH, value = f"//*[contains(@class, 'pagination-page page-item')][{podstran}]//a").click()
                        time.sleep(1)
                        print(f"Pobrana {podstran}. podstran")
                        dat.write(driver.page_source)
    

Koda, ki prebere lokalno shranjene html datoteke, ter karte shrani v csv in json format.


In [3]:
#debug_mode_za_branje_iz_lokalnih_datotek = False
path = 'Podatki o kartah/Podatki iz setov'
karte = []

# # Najprej poradiramo csv in json datoteki
with open("karte.csv", "w", encoding = "utf-8") as dat:
    pass

with open("karte.json", "w", encoding = "utf-8") as dat:
    pass

for filename in glob.glob(os.path.join(path, '*.html')):
    # Tukaj iz nekega razloga ne deluje utf-8, potrebno je latin1
    with open(filename, 'r', encoding = "latin1") as f:

        stetje = 0
        #print(f"Berem podatke kart iz {filename}")
        vsebina = f.read()
        
        for blok in vzorec_bloka.finditer(vsebina):
            stetje += 1
            
            # if debug_mode_za_branje_iz_lokalnih_datotek:
            #     print(stetje)
            #     print(blok.group(0))
            #     print(vzorec_karte.search(blok.group(0)).groupdict())
            
            karte.append(izlusci_podatke_o_kartah_iz_bloka(blok.group(0)))

with open("karte.json", "a", encoding = "utf-8") as dat:
    json.dump(karte, dat, indent=4, ensure_ascii=False)
    

with open("karte.csv", "a", encoding = "utf-8") as dat:
    writer = csv.DictWriter(dat, [
        "id_karte",
        "ime",
        "set",
        "redkost",
        "povprecna_cena",
        "povprecna_cena_foil",
    ])
    writer.writeheader()
    writer.writerows(karte)

print("Prebrali smo lokalno shanjene podatke iz HMTL datotek in jih zapisali v 'karte.csv' in 'karte.json'.")

Prebrali smo lokalno shanjene podatke iz HMTL datotek in jih zapisali v 'karte.csv' in 'karte.json'.


S pomočjo podatkov ustvarimo dataframe. Iz njega izločimo basic lande in tokene. Prav tako izločimo karte, ki nimajo podane vsaj ene cene.
*Opomba 1*: Ker imajo najbolj zaželene karte najbolj popolne podatke, nam tako izločanje ne bo pokvarilo vzorca.
*Opomba 2*: Nekateri seti so na strani MTGStocks navedeni napačno! Ker so mnogi izmed njih izredno pomembni, jih popravimo v dataframu. 

In [4]:
podatki_o_kartah = pd.read_csv("karte.csv", encoding="latin1")

regex_izraz_za_basic_lande = r'Forest \(\d|Mountain \(\d|Swamp \(\d|Plains \(\d|Island \(\d|Wastes \(\d'
okrajsani_podatki_o_kartah = podatki_o_kartah[~(podatki_o_kartah.redkost.isin(["Token", "Land"])) & ~(podatki_o_kartah.ime.str.match(regex_izraz_za_basic_lande)) & ~(podatki_o_kartah.ime.isin(["Forest", "Mountain", "Swamp", "Plains", "Island", "Wastes"]))]
okrajsani_podatki_o_kartah = okrajsani_podatki_o_kartah[~(okrajsani_podatki_o_kartah.povprecna_cena.isnull() & okrajsani_podatki_o_kartah.povprecna_cena_foil.isnull())]

slovar_za_popravljanje_napacnih_podatkov_s_spletne_strani = {'set': {"VAN" : "PVAN", "SLDC": "PHED", "XCLE": "CED", "XICE": "CEI", "RMB1": "FMB1", "XDND" : "AFR", "2E": "LEB", "2U": "2ED", "1E": "LEA", "PO2": "P02", "3ED":"FBB", "3E":"3ED"}}
okrajsani_podatki_o_kartah = okrajsani_podatki_o_kartah.replace(slovar_za_popravljanje_napacnih_podatkov_s_spletne_strani)

okrajsani_indeksi = okrajsani_podatki_o_kartah.id_karte # To so indeksi, s katerimi bomo pognali ustrezne url-je

raznoliki_seti = okrajsani_podatki_o_kartah["set"].unique()

print(okrajsani_podatki_o_kartah)


       id_karte               ime  set   redkost povprecna_cena  \
0         19748     Cyclonic Rift  RTR      Rare         $38.00   
1         19934   Utvara Hellkite  RTR    Mythic         $19.53   
2           360       Blood Crypt  RTR      Rare         $19.99   
3           404       Steam Vents  RTR      Rare         $18.63   
4           395    Overgrown Tomb  RTR      Rare         $13.99   
...         ...               ...  ...       ...            ...   
47532     18337  Dregscape Zombie  ARC    Common          $0.25   
47533     18380        Juggernaut  ARC  Uncommon          $0.25   
47534     18421           Plummet  ARC    Common          $0.25   
47535     18348          Fertilid  ARC    Common          $0.38   
47536     18434  Sanctum Gargoyle  ARC    Common          $0.24   

      povprecna_cena_foil  
0                  $57.99  
1                  $57.44  
2                  $34.37  
3                  $38.61  
4                  $25.72  
...                   ...  

Podatke o setih najdemo na strani "https://scryfall.com/sets".

In [5]:
# Spodnjo kocko poženemo samo v primeru, da podatki še niso lokalno shranjeni (oz. če spremenimo vrednost bool spremenljivke)
if podatki_s_spletnih_strani_se_niso_lokalno_shranjeni:
    driver = webdriver.Chrome(service = service_obj, options=chrome_options)
    url_do_strani_setov = "https://scryfall.com/sets"
    driver.get(url_do_strani_setov)
    time.sleep(1)
    vsebina_strani_setov = driver.page_source
    driver.close()

    slovarji_posameznih_setov = []

    with open("seti.csv", "w", encoding="utf-8") as csvfile:
        pass

    with open("seti.csv", "a", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, [
                "set",
                "polno_ime",
                "st_kart",
                "datum_izida",
            ])
        
        for posamezen_set in raznoliki_seti:
            vzorec_za_podatke_o_setu = re.compile(
                r'   (?P<polno_ime>\w.*?) <small>' + posamezen_set + r'</small>\n</a></td>.*?<td>\n.*?>(?P<st_kart>.*?)</a>\n.*?\n.*?\n.*?>(?P<datum_izida>.*?)</a>',
            )
            print("Shranjen", posamezen_set, end = " ")
            slovarji_posameznih_setov.append(izlusci_podatke_o_setih(vsebina_strani_setov, vzorec_za_podatke_o_setu, posamezen_set, podatki_o_kartah))
            
        writer.writeheader()
        writer.writerows(slovarji_posameznih_setov)
        
        with open("seti.json", "w", encoding = "utf-8") as dat:
            pass
        
        with open("seti.json", "a", encoding = "utf-8") as dat:
            json.dump(slovarji_posameznih_setov, dat, indent=4, ensure_ascii=False)

S pomočjo indeksov kart, ki smo jih dobili na strani MTGStocks, pridobimo podatke za vse karte iz dataframa.

In [10]:
driver = webdriver.Chrome(service = service_obj, options=chrome_options)
print("Število vseh indeksov kart, ki jih imamo:", len(okrajsani_indeksi))

# To je toliko da poradiramo
with open("specificne_lastnosti_kart.csv", "w", encoding = "utf-8") as csvfile:
    pass

with open("specificne_lastnosti_kart.csv", "a", encoding = "utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, [
        "id_karte",
        "cmc",
        "manacost",
        "barva",
        "povprecje_eu",
        "supertype",
        "cardtype",
        "subtype",
        "oracle_text",
        "reserved_list",
        "all_time_low",
        "all_time_low_datum",
        "all_time_high",
        "all_time_high_datum",
    ])
    
    slovarji_posameznih_kart = []
    stetje_zajetih_kart = 0

    for indeks_printa in okrajsani_indeksi[20000:22000]:
        url = f"https://www.mtgstocks.com/prints/{indeks_printa}"
        stetje_zajetih_kart += 1
        print(f"({stetje_zajetih_kart}.) Zajemanje printa karte št. {indeks_printa} s strani {url}")
        # with open(f"Podatki o kartah/Podatki o specificnih kartah/Podatki o karti st. {indeks_printa}.html", "w", encoding="utf-8") as dat:
        #try:
        driver.get(url)
        time.sleep(1)
        vsebina = driver.page_source
        
        glava_slovarja = {"id_karte": indeks_printa}
        manacost_in_barve = izlusci_podatke_manacosta_in_barve(vsebina)
        glavnina = izlusci_podatke_o_specificni_karti_iz_njene_datoteke(vsebina)
        glava_slovarja.update(manacost_in_barve)
        glava_slovarja.update(glavnina)
        print(glava_slovarja, "\n")
        slovarji_posameznih_kart.append(glava_slovarja)
            
        # except Exception as e:
        #     print(f"Prišlo je do napake pri setu {indeks_printa}")
        #     print(e)
            
    writer.writeheader()
    writer.writerows(slovarji_posameznih_kart)
        

Število vseh indeksov kart, ki jih imamo: 43057
(1.) Zajemanje printa karte št. 1695 s strani https://www.mtgstocks.com/prints/1695


C:\Users\jasak\AppData\Local\Temp\ipykernel_17804\4033016415.py:29: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  for indeks_printa in okrajsani_indeksi[100:110]:


Instant
{'id_karte': 1695, 'cmc': 1, 'manacost': 'w', 'barva': 'w', 'povprecje_eu': '€0.47', 'supertype': [], 'cardtype': ['Instant'], 'subtype': [], 'oracle_text': 'Choose a color. White creatures you control gain protection from the chosen color until end of turn.', 'reserved_list': 'No', 'all_time_low': '$0.27', 'all_time_low_datum': 'Mar 17, 2019', 'all_time_high': '$0.65', 'all_time_high_datum': 'Jun 11, 2014'} 

(2.) Zajemanje printa karte št. 1706 s strani https://www.mtgstocks.com/prints/1706
Creature - Merfolk Wizard, 
{'id_karte': 1706, 'cmc': 1, 'manacost': 'u', 'barva': 'u', 'povprecje_eu': '€0.45', 'supertype': [], 'cardtype': ['Creature'], 'subtype': ['Merfolk', 'Wizard'], 'oracle_text': "Whenever an opponent shuffles his or her library, you may put a +1/+1 counter on Cosi's Trickster.", 'reserved_list': 'No', 'all_time_low': '$0.28', 'all_time_low_datum': 'Mar 27, 2013', 'all_time_high': '$0.90', 'all_time_high_datum': 'Aug 8, 2015'} 

(3.) Zajemanje printa karte št. 188

# Spodaj se nahajajo testni bloki

In [7]:
string_za_poskus = ''
print(re.findall(vzorec_za_manacost, string_za_poskus))

print(izlusci_podatke_manacosta_in_barve(string_za_poskus))

[]
{'cmc': 0, 'manacost': '0', 'barva': 'c'}


In [8]:
testni_string = '<i class="ms ms-loyalty-1 ms-loyalty-up"></i>: Until your next turn, whenever a creature deals combat damage to Vraska the Unseen, destroy that creature.\n<i class="ms ms-loyalty-3 ms-loyalty-down"></i>: Destroy target nonland permanent.\n<i class="ms ms-loyalty-7 ms-loyalty-down"></i>: Put three 1/1 black Assassin creature tokens onto the battlefield with "Whenever this creature deals combat damage to a player, that player loses the game."'
with open("testni_zajcek.txt", "r", encoding = "utf-8") as testna_datoteka:
    testni_string = testna_datoteka.read()

popravljen_string = re.sub(vzorec_za_popravo_oracle_texta_notranje_znacke, r"(\1)", testni_string)
popravljen_string = re.sub(vzorec_za_popravo_oracle_texta_zunanje_znacke, r"\1", popravljen_string)
popravljen_string = re.sub(vzorec_za_popravo_oracle_texta_planeswalker_uptake, r"(+\1)", popravljen_string)
popravljen_string = re.sub(vzorec_za_popravo_oracle_texta_planeswalker_downtake, r"(-\1)", popravljen_string)
popravljen_string = popravljen_string.replace('""(t)',"(tap)") # To je zaradi buga na spletni strani
print(popravljen_string)

blabllablablbla


In [9]:
def doloci_super_sub_in_cardtype(niz):
    niz = niz.removesuffix(", ").strip()
    supertype, subtype, cardtype = [], [], []
    
    # najprej polovimo nekaj starih karth (niso navedene pravilno na spletni strani)
    if "Enchant " in niz:
        cardtype.append("Enchantment")
        subtype.append("Aura")
        return supertype, cardtype, subtype
    if "Summon " in niz:
        niz = niz.replace("Summon ", "Creature - ")
        if "Legend" in niz:
            supertype.append("Legendary")
            niz = niz.replace("Legend", "")
    
    # Če smo pri tem izpraznili niz, samo returnamo
    if len(niz) > 0:
        for suprtype in seznam_supertypov:
            if suprtype in niz:
                supertype.append(suprtype)
                niz = niz.replace(f"{suprtype} ","")
        if "-" in niz or "—" in niz:
            niz = niz.replace("—","-")
            str_cardtype, str_subtype = niz.split(" - ")
            cardtype = [typ.strip() for typ in str_cardtype.split(" ") if typ.strip() != ""]
            subtype = [typ.strip() for typ in str_subtype.split(" ") if typ.strip() != ""]
        elif " " in niz:
            cardtype = [typ.strip() for typ in niz.split(" ") if typ.strip() != ""]
        else:
            cardtype.append(niz.strip())
        
    return supertype, cardtype, subtype

testni_string = "Legendary Planeswalker — Jace, "
testni_string = "Enchant Player"

print(doloci_super_sub_in_cardtype(testni_string))

    

([], ['Enchantment'], ['Aura'])
